# Customer RFM Segmentation
Customer segmentation used to target shipping cost voucher receiver candidate. As stated on my analysis before, the use of voucher doesn't seem to have any effect on user's satisfaction. However, this strategy may be able to increase user retention. Since the majority of users only order at most 2 times in the span of 2 years this is a severe problem for the ecommerce. Therefore a strategy needed to be laid out to increase user transaction frequency and monetary value.

## Free Shipping Voucher Strategy
Shipping cost voucher will be awarded to customer which has lower transaction frequency but relatively higher monetary value. Not only that, but customer will be able to use the voucher only after it fulfill some conditions such as minimum transaction value, applied only to Official Store, or on certain occassion or date.

## Library Import and Configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# import postgres connection factory utility
from util.dbconn import DbConn

In [ ]:
WAREHOUSE_CONN = {
    'user': 'postgres',
    'password': 'qshOke46RvOg0',
    'host': '10.128.0.2',
    'port': '8084',
    'dbname': 'postgres'
}

In [ ]:
postgres_conn = DbConn(WAREHOUSE_CONN).get_conn()

## Data Overview

In [ ]:
customer_rfm_query = """SELECT 
	ud.user_sk,
	mcmv.total_spending,
	mcmv.total_shipping_cost,
	mcof.count_order,
	EXTRACT(epoch FROM mcr.last_order_interval/86400)::int8 AS last_order_days
FROM 
	warehouse.user_dim ud 
LEFT JOIN warehouse.mart_customer_monetary_value mcmv ON
	ud.user_sk = mcmv.user_sk
LEFT JOIN warehouse.mart_customer_order_frequency mcof ON
	ud.user_sk = mcof.user_sk
LEFT JOIN warehouse.mart_customer_recency mcr ON
	ud.user_sk = mcr.user_sk;"""
customer_rfm = pd.read_sql(customer_rfm_query, con=postgres_conn)
display(customer_rfm.info())
display(customer_rfm.head(5))
display(customer_rfm.describe())

### Check for Null Value

In [ ]:
customer_rfm.isnull().sum()

## Exploratory Data Analysis

### Order Frequency

In [ ]:
fig, ax = plt.subplots()
t1 = customer_rfm.loc[:, ['user_sk', 'count_order']] \
        .groupby('count_order', as_index=False).count()
ax.bar(t1['count_order'], t1['user_sk'])
ax.set_title('Order Frequency Count')
ax.set_xlabel('Order Frequency')
ax.set_ylabel('Order Count')

display(t1)
plt.show()

### Order Recency

In [ ]:
fig, ax = plt.subplots()
t1 = customer_rfm.loc[:, ['user_sk', 'last_order_days']] \
        .groupby('last_order_days', as_index=False).count()
ax.bar(t1['last_order_days'], t1['user_sk'])
ax.set_title('Days Since Last Order Distribution')
ax.set_xlabel('Last Order Days')
ax.set_ylabel('Order Count')

plt.show()
t1.columns = ['days_since_last_order', 'count_order']
display(t1.head(15))

### Monetary Value

liat data mentahnya dulu <br>
pake bar chart 

In [ ]:
fig, ax = plt.subplots()

t1 = customer_rfm.loc[:, ['user_sk', 'total_spending']] \
        .groupby('total_spending', as_index=False).count()

# ax.bar(t1['total_spending'], t1['user_sk'])
# ax.set_title('Customer Monetary Value')
# ax.set_xlabel('Total Spending')
# ax.set_ylabel('Order Count')

# plt.show()
t1.columns = ['total_spending', 'count_order']
display(t1.head(15))

In [ ]:
fig, ax = plt.subplots()

t0 = customer_rfm.copy()

t0['monetary_value'] = pd.qcut(customer_rfm['total_spending'], q=100, labels=False, duplicates='drop')
t1 = t0.loc[:, ['user_sk', 'monetary_value']] \
        .groupby('monetary_value', as_index=False).count()
ax.bar(t1['monetary_value'], t1['user_sk'])
ax.set_title('Monetary Value')
ax.set_xlabel('Total Spending')
ax.set_ylabel('User Count')

display(t1)
plt.show()

## Baseline Model
Manually assigned RFM score based on percentile.
<table>
    <tr>
        <th>Recency</th>
        <th>Frequency</th>
        <th>Monetary</th>
    </tr>
    <tr>
        <td>R-Tier-1 (most recent)</td>
        <td>F-Tier-1 (most frequent)</td>
        <td>M-Tier-1 (highest spend)</td>
    </tr>
    <tr>
        <td>R-Tier-2</td>
        <td>F-Tier-2</td>
        <td>M-Tier-2</td>
    </tr>
    <tr>
        <td>R-Tier-3</td>
        <td>F-Tier-3</td>
        <td>M-Tier-3</td>
    </tr>
    <tr>
        <td>R-Tier-4 (least recent)</td>
        <td>F-Tier-4 (least frequent)</td>
        <td>M-Tier-4 (lowest spend)</td>
    </tr>
</table>
Reiterating the goals again, the strategy prefer customer which has lower frequency but relatively higher monetary value. Below are the segment name assignment based on customer RFM score:<br>
<table>
<thead>
  <tr>
    <th>R</th>
    <th>F</th>
    <th>M</th>
    <th>Customer Segment</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>1-2</td>
    <td>1-4</td>
    <td>1-2</td>
    <td>High-spending Active Customer **</td>
  </tr>
  <tr>
    <td>1-2</td>
    <td>1-4</td>
    <td>3-4</td>
    <td>Low-spending Active Customer *</td>
  </tr>
  <tr>
    <td>3-4</td>
    <td>1-4</td>
    <td>1-2</td>
    <td>Churned High-spending Customer **</td>
  </tr>
  <tr>
    <td>3-4</td>
    <td>1-4</td>
    <td>3-4</td>
    <td>Churned Low-spending Customer *</td>
  </tr>
</tbody>
</table>
<b>Note:</b><br>
** The best candidate to receive free shipping voucher is marked with double asterisk<br>
* The second best candidate is marked with single asterisk

In [ ]:
rfm = customer_rfm.copy()
rfm.info()

### Drop Null Values

In [ ]:
rfm = rfm.dropna()
rfm.isnull().sum()

## Assign RFM Tier

### Recency Tier

In [ ]:
rfm['recency_tier'] = pd.qcut(rfm['last_order_days'], q=4, labels=range(1, 5))

fig, ax = plt.subplots()

t1 = rfm.loc[:, ['user_sk', 'recency_tier']] \
        .groupby('recency_tier', as_index=False).count()
ax.bar(t1['recency_tier'], t1['user_sk'])
ax.set_title('Customer Recency Tier')
ax.set_xlabel('Recency Tier')
ax.set_ylabel('User Count')

display(t1)
plt.show()

### Frequency Tier

In [ ]:
rfm.loc[(rfm['count_order'] >= 5) & (rfm['count_order'] <= 16), 'frequency_tier'] = 1
rfm.loc[(rfm['count_order'] >= 3) & (rfm['count_order'] <= 4), 'frequency_tier'] = 2
rfm.loc[rfm['count_order'] == 2, 'frequency_tier'] = 3
rfm.loc[rfm['count_order'] == 1, 'frequency_tier'] = 4

rfm['frequency_tier'] = rfm['frequency_tier'].astype('int')

fig, ax = plt.subplots()

t1 = rfm.loc[:, ['user_sk', 'frequency_tier']] \
        .groupby('frequency_tier', as_index=False).count()
ax.bar(t1['frequency_tier'], t1['user_sk'])
ax.set_title('Customer Frequency Tier')
ax.set_xlabel('Frequency Tier')
ax.set_ylabel('User Count')

display(t1)
plt.show()

### Monetary Tier

In [ ]:
# cek pake selain pd.cut
rfm['monetary_tier'] = pd.qcut(rfm['total_spending'], q=4, labels=range(4, 0, -1))

fig, ax = plt.subplots()

t1 = rfm.loc[:, ['user_sk', 'monetary_tier']] \
        .groupby('monetary_tier', as_index=False).count()
ax.bar(t1['monetary_tier'], t1['user_sk'])
ax.set_title('Monetary Tier')
ax.set_xlabel('Monetary Tier')
ax.set_ylabel('User Count')

display(t1)
plt.show()

In [ ]:
rfm.info()

## Distribution of RFM Tier

In [ ]:
tt1 = rfm.copy()
tt1['frequency_tier'] = tt1.apply(lambda row: row['frequency_tier']+(np.random.rand()*0.3), axis=1)
tt1['monetary_tier'] = tt1.apply(lambda row: row['monetary_tier']+(np.random.rand()*0.3), axis=1)
tt1['recency_tier'] = tt1.apply(lambda row: row['recency_tier']+(np.random.rand()*0.3), axis=1)

t = ['frequency_tier', 'monetary_tier', 'recency_tier']
c = []
f = []

for i in range(0, len(t)):
    for j in range(0, len(t)):
        t0 = t[i]
        t1 = t[j]
        
        if t0 == t1:
            continue
        
        if [t0, t1] in c:
            continue
        else:
            f.append([t0, t1])
            c.append([t0, t1])
            c.append([t1, t0])

fig, axs = plt.subplots(len(f), 1)
fig.set_size_inches(5, 15, forward=True)

for i in range(0, len(f)):
    ax = axs[i]
    ax.scatter(tt1[f[i][0]], tt1[f[i][1]])
    ax.set_title('{} vs {}'.format(f[i][0], f[i][1]))
    ax.set_xlabel(f[i][0])
    ax.set_ylabel(f[i][1])

plt.tight_layout()
plt.show()

## Customer Segmentation

### High Spending Active Customer

In [ ]:
high_spending_active_cust_cond = \
    (rfm['recency_tier'] <= 2) \
    & (rfm['monetary_tier'] >= 2)

high_spending_active_cust = rfm[high_spending_active_cust_cond]
display(high_spending_active_cust.head(10))
display(high_spending_active_cust.shape)

rfm.loc[high_spending_active_cust_cond, 'customer_segment'] = 'hsac'

### Low Spending Active Customer

In [ ]:
low_spending_active_cust_cond = \
    (rfm['recency_tier'] <= 2) \
    & (rfm['monetary_tier'] <= 3)

low_spending_active_cust = rfm[low_spending_active_cust_cond]
display(low_spending_active_cust.head(10))
display(low_spending_active_cust.shape)

rfm.loc[low_spending_active_cust_cond, 'customer_segment'] = 'lsac'

### Churned High Spending Customer

In [ ]:
churned_high_spending_cust_cond = \
    (rfm['recency_tier'] >= 3) \
    & (rfm['monetary_tier'] >= 2)

churned_high_spending_cust = rfm[churned_high_spending_cust_cond]
display(churned_high_spending_cust.head(10))
display(churned_high_spending_cust.shape)

rfm.loc[churned_high_spending_cust_cond, 'customer_segment'] = 'chsc'

### Churned Low Spending Customer

In [ ]:
churned_low_spending_cust_cond = \
    (rfm['recency_tier'] >= 3) \
    & (rfm['monetary_tier'] <= 3)

churned_low_spending_cust = rfm[churned_low_spending_cust_cond]
display(churned_low_spending_cust.head(10))
display(churned_low_spending_cust.shape)

rfm.loc[churned_low_spending_cust_cond, 'customer_segment'] = 'clsc'

### Check Segment For All Customer

In [ ]:
rfm.isnull().sum()

### Base Model Cluster Result

In [ ]:
rfm['monet_code'] = 4 - rfm['monetary_tier'].astype('category').cat.codes

In [ ]:
rfm.loc[:, ['monetary_tier', 'monet_code']]

In [ ]:
rfm['recency_tier'] = pd.factorize(rfm['recency_tier'], sort=True)[0] + 1
rfm['monetary_tier'] = 4 - rfm['monetary_tier'].astype('category').cat.codes

hsac = rfm[rfm['customer_segment'] == 'hsac']
lsac = rfm[rfm['customer_segment'] == 'lsac']
chsc = rfm[rfm['customer_segment'] == 'chsc']
clsc = rfm[rfm['customer_segment'] == 'clsc']

hsac = hsac.loc[:, ['recency_tier', 'frequency_tier', 'monetary_tier']]
lsac = lsac.loc[:, ['recency_tier', 'frequency_tier', 'monetary_tier']]
chsc = chsc.loc[:, ['recency_tier', 'frequency_tier', 'monetary_tier']]
clsc = clsc.loc[:, ['recency_tier', 'frequency_tier', 'monetary_tier']]

In [ ]:
def add_random_noise(s, noise_strength=0.5):
    np.random.seed(np.random.randint(1, len(s)))
    noise = [np.random.rand() * (np.random.rand() * noise_strength) for i in range(0, len(s))]
    s = s + noise
    return s

rfm_t = rfm.copy()
rfm_t['recency_tier'] = add_random_noise(rfm_t.loc[:, 'recency_tier'], 0.3)
rfm_t['frequency_tier'] = add_random_noise(rfm_t.loc[:, 'frequency_tier'], 0.3)
rfm_t['monetary_tier'] = add_random_noise(rfm_t.loc[:, 'monetary_tier'], 0.3)

colors = ['r', 'limegreen', 'b', 'orange']
segment = ['hsac', 'lsac', 'chsc', 'clsc']

t = ['frequency_tier', 'monetary_tier', 'recency_tier']
c = []
f = []

for i in range(0, len(t)):
    for j in range(0, len(t)):
        t0 = t[i]
        t1 = t[j]
        
        if t0 == t1:
            continue
        
        if [t0, t1] in c:
            continue
        else:
            f.append([t0, t1])
            c.append([t0, t1])
            c.append([t1, t0])

fig, axs = plt.subplots(len(f), 1)
fig.set_size_inches(5, 15, forward=True)

for i in range(0, len(f)):
    for j in segment:
        ax = axs[i]
        _ttt = rfm_t[rfm_t['customer_segment'] == j]
        ax.scatter(_ttt[f[i][0]], _ttt[f[i][1]], color=colors[segment.index(j)])
        ax.set_title('{} vs {}'.format(f[i][0], f[i][1]))
        ax.set_xlabel(f[i][0])
        ax.set_ylabel(f[i][1])

plt.tight_layout()
plt.show()

### Baseline Model Evaluation

In [ ]:
from sklearn.metrics import silhouette_score

rfm_eval = rfm.loc[:, ['recency_tier', 'frequency_tier', 'monetary_tier', 'customer_segment']]

rfm_eval.loc[rfm_eval['customer_segment'] == 'hsac', 'customer_segment_code'] = 0
rfm_eval.loc[rfm_eval['customer_segment'] == 'lsac', 'customer_segment_code'] = 1
rfm_eval.loc[rfm_eval['customer_segment'] == 'chsc', 'customer_segment_code'] = 2
rfm_eval.loc[rfm_eval['customer_segment'] == 'clsc', 'customer_segment_code'] = 3

silhouette_score(rfm_eval.loc[:, ['recency_tier', 'frequency_tier', 'monetary_tier']], \
                 rfm_eval.loc[:, 'customer_segment_code'])

## Data Preprocessing

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = customer_rfm.copy()
df = df.loc[:, ['last_order_days', 'count_order', 'total_spending']]

In [ ]:
df = df.dropna()

In [ ]:
# TODO: more preprocessing proper scaling

In [ ]:
mmscaler = MinMaxScaler()
df = pd.DataFrame(mmscaler.fit_transform(df))
df.columns = ['last_order_days', 'count_order', 'total_spending']

In [ ]:
display(df.head(10))

## Model Selection

In [ ]:
from sklearn.cluster import KMeans

### Using Silhouette Score to get optimal cluster

In [ ]:
# df1 = df.copy()

# n_cluster = []
# silhouette_scores = []

# for i in range(2,4):
#     print('Training model ...')
#     kmeans = KMeans(n_clusters=i, random_state=111)
#     y = kmeans.fit_predict(df1)
#     print('Calculating silhouette score ...')
#     sc = silhouette_score(df1, y)
#     n_cluster.append(i)
#     silhouette_scores.append(sc)
#     print('For n_cluster = {n} The avg of silhouette score is {score}'.format(n=i, score=sc))

# fig, ax = plt.subplots()

# ax.plot(n_cluster, silhouette_scores)
# ax.set_xlabel('n Cluster')
# ax.set_ylabel('Silhouette score')

# plt.show()

In [ ]:
df1 = df.copy()

n_cluster = []
inertia_values = []

for i in range(2,7):
    kmeans = KMeans(n_clusters=i, random_state=111)
    y = kmeans.fit_predict(df1)
    n_cluster.append(i)
    inertia_values.append(kmeans.inertia_)

fig, ax = plt.subplots()

ax.plot(n_cluster, inertia_values)
ax.set_title('Within cluster sum of squared distance vs cluster size')
ax.set_xlabel('n Cluster')
ax.set_ylabel('Sum of squared distance to cluster center')

plt.show()

## K-Means

In [ ]:
kmeans = KMeans(n_clusters=4)
y = kmeans.fit_predict(df1)

### K-Means Evaluation

In [ ]:
kmeans.inertia_

In [ ]:
def add_random_noise(s, noise_strength=0.5):
    np.random.seed(np.random.randint(1, len(s)))
    noise = [np.random.rand() * (np.random.rand() * noise_strength) for i in range(0, len(s))]
    s = s + noise
    return s

kmeans_df = df.copy()

colors = ['r', 'limegreen', 'b', 'orange']
segment = [i for i in range(0, 4)]

# Set label to predicted cluster
kmeans_df['label'] = y

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in segment:
    _df = kmeans_df[kmeans_df['label'] == i]
    xs = _df.loc[:, 'last_order_days']
    ys = _df.loc[:, 'count_order']
    zs = _df.loc[:, 'total_spending']
    ax.scatter(xs, ys, zs, color=colors[i])

ax.set_xlabel('last_order_days')
ax.set_ylabel('count_order')
ax.set_zlabel('total_spending')

plt.show()

## K-Medoids

In [ ]:
from sklearn_extra.cluster import KMedoids

In [ ]:
kmedoids = KMedoids(n_clusters=4)
y = kmedoids.fit_predict(df1)

## Hierarchical Agglomerative

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
df1 = df.copy()

agglomerative_clustering = AgglomerativeClustering(
                                n_clusters=4)
y = agglomerative_clustering.fit_predict(df1)

In [ ]:
def add_random_noise(s, noise_strength=0.5):
    np.random.seed(np.random.randint(1, len(s)))
    noise = [np.random.rand() * (np.random.rand() * noise_strength) for i in range(0, len(s))]
    s = s + noise
    return s

agglomerative_df = df.copy()

colors = ['r', 'limegreen', 'b', 'orange']
segment = [i for i in range(0, 4)]

# Set label to predicted cluster
agglomerative_df['label'] = y

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in segment:
    _df = agglomerative_df[agglomerative_df['label'] == i]
    xs = _df.loc[:, 'last_order_days']
    ys = _df.loc[:, 'count_order']
    zs = _df.loc[:, 'total_spending']
    ax.scatter(xs, ys, zs, color=colors[i])

ax.set_xlabel('last_order_days')
ax.set_ylabel('count_order')
ax.set_zlabel('total_spending')

plt.show()

In [ ]:
ac_pca = PCA(n_components=2)
ac_df = df1.copy()
ac_df = ac_pca.fit_transform(ac_df)

ac_df = pd.DataFrame(ac_df)
ac_df['label'] = y

cluster_1 = ac_df[ac_df['label'] == 0]
cluster_2 = ac_df[ac_df['label'] == 1]

fig, ax = plt.subplots()

ax.scatter(cluster_1.iloc[:, 0], cluster_1.iloc[:, 1], color='r')
ax.scatter(cluster_2.iloc[:, 0], cluster_2.iloc[:, 1], color='b')
ax.set_xlabel('PCA Feature 1')
ax.set_ylabel('PCA Feature 2')
ax.set_title('Agglomerative Clustering')

plt.show()

**visualisasi per 2 fitur utk tampilin semuanya